
# A case study on 1994 US income census data 

## Table of Contents
* [Introduction](#section-1)
* [Importing necessary libraries and packages](#section-2)
* [Understanding the data](#section-3)
    - [Importing the dataset](#subsect-1)
    - [Preliminary Analysis](#subsect-2) 
    - [Understanding categorical variables](#subsect-3)  
* [Exploratory data analysis](#section-4)    
    - [Analysis of the effect of different variables on 'target' variable](#subsect-4) 
    - [Splitting the dataframe with respect to capital transactions and analysing](#subsect-5)
    - [Correlation Analysis](#subsect-6)
* [Building the ML model on zero_cap_df [for those with zero capital transactions]](#section-5)
    - [Model building with Recursive Feature Elimination](#subsect-7)   
    - [Random forest with RFE](#subsect-8) 
    - [Model building with Principal Component Analysis (PCA)](#subsect-9)  
    - [Random forest with PCA](#subsect-10)
    - [Gradient boosting method with RFE](#subsect-11)
* [Building the ML model on gain_loss_cap_df [for those with some significant capital transactions]](#section-6)
    - [Random forest with RFE](#subsect-12) 
* [Conclusions](#section-7)

<a id="section-1"></a>
# Introduction

#### About the dataset
The dataset has been taken from the famous UCI Machine Learning Repository. Extraction was done by Barry Becker from the 1994 Census database. The dataset is set for a prediction task to determine whether a person makes over $50,000/year.

#### Business objective
* To understand and arrive at some interesting insights related to income of an individual from the individual's work, education and personal details

* To buid a predictive model to serve two main purposes:
>* To predict whether an individual falls to an annual income category of above $50,000, with known information such as age, workclass, education, etc.
>* To identify the important variables that strongly influence the prediction
##### Procedure
* First objective can be extracted by data visualization achieved through exploratory data analysis

* Second objective is achieved by building various machine learning models optimized for better results

#### Expected Outcomes from the business objectives
* Based on the income category an individual falls into and understanding the income pattern of certain groups, goa targeted marketing campaigns can be carried out to reach out to such groups and hence individuals.

* By knowing the features that influence the high income of an individual, companies and governments get an estimate so as to understand to which income slab does an employee fit into.

##### Attribute Information

* 'target' variable: to classify the income of individual into >$50K or <=$50K category.

>* age: continuous.
>* workclass: Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked.
>* fnlwgt: continuous.
>* education: Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool.
>* education-num: continuous.
>* marital-status: Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse.
>* occupation: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces.
>* relationship: Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried.
>* race: White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black.
>* sex: Female, Male.
>* capital-gain: continuous.
>* capital-loss: continuous.
>* hours-per-week: continuous.
>* native-country: United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands.

<a id="section-2"></a>
# Importing necessary libraries and packages

In [2]:
!pip install pydotplus
!pip install graphviz

In [94]:
# Filtering out the warnings

import warnings
warnings.filterwarnings('ignore')

# Importing the necessary libraries for exploratory data analysis

import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# Importing the necessary machine learning libraries

# Test train split
from sklearn.model_selection import train_test_split
# Preprocessing
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.preprocessing import LabelEncoder

# under/over sampling
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE, RFECV
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.decomposition import PCA, IncrementalPCA

# Metrics
from sklearn.metrics import confusion_matrix, roc_curve, roc_auc_score, accuracy_score
from sklearn.metrics import precision_recall_curve, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report
from imblearn.metrics import sensitivity_specificity_support
from sklearn.metrics import plot_roc_curve
from sklearn.metrics import log_loss

# pipelines
from sklearn.pipeline import Pipeline
from imblearn.pipeline import make_pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin

# Cross validation
from sklearn.model_selection import StratifiedKFold, RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

# Tree based algorithms
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

# Boosting algorithms
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier

# Importing required packages for visualization
from IPython.display import Image  
from six import StringIO  
from sklearn.tree import export_graphviz
import pydotplus, graphviz
import plotly.offline as py
import plotly.graph_objs as go

In [95]:
# Code to display maximum rows and columns 

pd.set_option("display.max_rows", None, "display.max_columns", None)

<a id="section-3"></a>
# Understanding the data

<a id="subsect-1"></a>
#### Importing the dataset

In [96]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [187]:
# Importing the dataset and assigning it to income census dataframe (icd_df)

file_path = "/kaggle/input/adult-census-income/adult.csv"
icd_df = pd.read_csv(file_path, delimiter=",")
icd_df.head()

In [188]:
# Assigning column names

icd_df.columns = ['age','workclass','fnlwgt','education','education_num','marital_status','occupation','relationship','race','sex','capital_gain','capital_loss','hours_per_week','native_country','target']
icd_df.head(2)

<a id="subsect-2"></a>
#### Preliminary analysis 
Data handling, data cleaning, checking outliers, etc

In [189]:
icd_df.info(verbose=1)

#### Observations
* Data set contains 32561 rows/individuals and 14 dependent features with one target variable
* The datatypes of the features are a mixture of object and int 

In [190]:
icd_df.describe()

#### Observations
* age of individuals lies in bracket of 17 to 90 years, with a mean age of ~38 years
* number of educational years is in a bracket of 1-16 years, with mean at 10 years
* Average working hours per week was around 40 hrs 
#### Possible outliers in 'capital_gain' and 'capital_loss', huge difference between mean and median in these two variables is observed

In [191]:
icd_df.capital_gain.describe([0.5,0.75,0.91,0.92,0.99])

#### Observations
* About 91% of the 32561 individuals have 0 capital gains, while the top 9% individuals have some capital gains.
* Top 1% (individuals in greater than 99th percentile bracket) have significantly higher capital gains, making the distribution skewed.

In [192]:
icd_df.capital_loss.describe([0.5,0.75,0.95,0.96,0.99])

#### Observations
* About 95% of the 32561 individuals have 0 capital gains, while the top 5% individuals have some capital loss.
* Top 1% (individuals in greater than 99th percentile bracket) have significantly higher capital loss, making the distribution skewed.

In [193]:
icd_df[icd_df.capital_gain != 0].shape

In [194]:
icd_df[icd_df.capital_loss != 0].shape

In [195]:
icd_df[(icd_df.capital_gain != 0) & (icd_df.capital_loss != 0)].shape

#### Observations
* There are around 2712 individuals out of 32561, who have significant capital gains
* Around 1519 individuals out of 32561, have significant loss in capital
* Interestingly the individuals having capital gains and capital loss are different people.
* So around 4231 individuals have some gain or loss in capital, which is around 13 % of the population

##### Dropping 13% of the data can have adverse effect on the analysis. Hence the dataset can be split into two dataframes, one for those who have some significant capital transactions and those with zero capital transactions 

In [196]:
# Classifying columns

cat_cols = ['workclass','education','marital_status','occupation','relationship','race','sex','native_country']

num_cols = ['age','fnlwgt','education_num','capital_gain','capital_loss','hours_per_week']

In [197]:
# Percentage of null values in each columns 

round(100*icd_df.isnull().sum()/len(icd_df),2).sort_values(ascending = False)

From the above output, there seems to be no missing values

<a id="subsect-3"></a>
### Understanding categorical variables

In [198]:
cat_cols

##### Understanding value counts of each variables

In [199]:
icd_df.workclass.value_counts(normalize = True,dropna=False)*100

In [200]:
icd_df.education.value_counts(normalize = True,dropna=False)*100

In [201]:
icd_df.occupation.value_counts(normalize = True,dropna=False)*100

In [202]:
icd_df.marital_status.value_counts(normalize = True,dropna=False)*100

In [203]:
icd_df.relationship.value_counts(normalize = True,dropna=False)*100

In [204]:
icd_df.race.value_counts(normalize = True,dropna=False)*100

In [205]:
icd_df.sex.value_counts(normalize = True,dropna=False)*100

In [206]:
icd_df.native_country.value_counts(normalize = True,dropna=False)*100

In [207]:
icd_df.target.value_counts(normalize = True,dropna=False)*100

##### Though the null value count is zero, there are lot of '?' values in 'workclass', 'occupation' and 'native_country'  columns indicating missing values

##### For categorical columns, missing value imputation is most popularly done using mode imputation 

In [208]:
icd_df[icd_df['workclass'] == '?'].shape

In [209]:
icd_df[icd_df['occupation'] == '?'].shape

In [210]:
icd_df[icd_df['native_country'] == '?'].shape

In [211]:
# Missing value imputation

for col in ['workclass', 'occupation', 'native_country']:
    icd_df[col] = icd_df[col].replace('?', icd_df[col].mode()[0])

In [212]:
icd_df.workclass.value_counts(normalize = True,dropna=False)*100

In [213]:
icd_df.occupation.value_counts(normalize = True,dropna=False)*100

In [214]:
icd_df.native_country.value_counts(normalize = True,dropna=False)*100

 #### Missing values have been handled

In [215]:
icd_df.head()

<a id="section-4"></a>
# Exploratory data analysis 

#### Check for Data imbalance in 'target' variable

In [216]:
icd_df.target.value_counts(normalize=True)*100

##### * The dataset is not balanced, around 3/4th of the sample population have an income of less than or equal to $50,000/year 

<a id="subsect-4"></a>
### Analysis of the effect of different variables on 'target' variable

#### with respect to 'sex'

In [217]:
fig = plt.figure(figsize=(8,5))
sns.countplot(x ='sex', hue = "target", data = icd_df, palette = 'Blues')
plt.show()

#### Observations
* The proportion of low income group outnumber the high income group significantly
* The proportional difference in low-to-high income groups is quite large for female, while the ratio is ~10:1 for female, it is ~2:1 for male

#### with respect to 'workclass'

In [218]:
fig = plt.figure(figsize=(10,5))
sns.countplot(x ='workclass', hue = "target", data = icd_df, palette = 'Blues')
plt.show()

#### Observations
* Individuals working in private companies are a large chunk of people have higher income
* Among all the working class excepte for self-emp-inc, people with low income category outnumber the high income group 

#### with respect to 'occupation'

In [219]:
fig = plt.figure(figsize=(8,15))
sns.countplot(y ='occupation', hue = "target", data = icd_df, orient= 'v', palette = 'Blues')
plt.show()

#### Observations
* People in professional speciality and executive managerial roles have relatively higher income

#### with respect to 'education'

In [220]:
# For the purpose of anlysis, school education is merged 
df_refere = icd_df.copy(deep=True)
df_refere['education'] = df_refere['education'].replace([' Preschool',' 1st-4th',' 5th-6th',' 7th-8th',
                                                        ' 9th',' 10th',' 11th',' 12th'],'school_education')

fig = plt.figure(figsize=(8,15))
sns.countplot(y ='education', hue = "target", data = df_refere, orient= 'v', palette = 'Blues')
plt.show()

#### Observations
* People with some form of bachelor degree or above are among significant population earning high income

#### with respect to 'relationship'

In [221]:
fig = plt.figure(figsize=(8,12))
sns.countplot(y ='relationship', hue = "target", data = icd_df, orient= 'v', palette = 'Blues')
plt.show()

#### Observations
* Individuals with single status earn low income compared to their married counterparts 

#### with respect to 'race'

In [222]:
fig = plt.figure(figsize=(8,15))
sns.countplot(y ='race', hue = "target", data = icd_df, orient= 'v', palette = 'Blues')
plt.show()

#### Observations
* Whites are among a large chunk of people who earn high income and they are also among large chunk of low income earners 

In [223]:
# Splitting the dataframe into low income and high income dataframes for the purpose of analysis

low_income_df = icd_df[icd_df['target']=='<=50K']
high_income_df = icd_df[icd_df['target']=='>50K']

print('Shape of low_income_df: ',low_income_df.shape)
print('Shape of high_income_df: ',high_income_df.shape)

In [224]:
# Grouping low income earners among different races

low_inc_cnt_race_wise = pd.DataFrame(low_income_df.groupby('race').size())
low_inc_cnt_race_wise.columns = ['low_income_count']
print(low_inc_cnt_race_wise.shape)
low_inc_cnt_race_wise.head()

In [225]:
# Grouping high income earners among different races

high_inc_cnt_race_wise = pd.DataFrame(high_income_df.groupby('race').size())
high_inc_cnt_race_wise.columns = ['high_income_count']
print(high_inc_cnt_race_wise.shape)
high_inc_cnt_race_wise.head()

In [226]:
# Merging the above two dataframes

inc_cnt_race_wise = low_inc_cnt_race_wise.merge(high_inc_cnt_race_wise, on = 'race', how = 'left')
inc_cnt_race_wise['percent_low_income'] = ((inc_cnt_race_wise['low_income_count']*100)/(inc_cnt_race_wise['low_income_count']+inc_cnt_race_wise['high_income_count'])).round(2)
inc_cnt_race_wise['percent_high_income'] = ((inc_cnt_race_wise['high_income_count']*100)/(inc_cnt_race_wise['low_income_count']+inc_cnt_race_wise['high_income_count'])).round(2)
inc_cnt_race_wise.loc[:,['percent_low_income','percent_high_income']]

In [227]:
fig = plt.figure(figsize=(12,6))

plt.subplot(121)
inc_cnt_race_wise.percent_low_income.plot.bar()
plt.title("low_income")
plt.grid(axis = 'y')

plt.subplot(122)
inc_cnt_race_wise.percent_high_income.plot.bar()
plt.title("high_income")
plt.grid(axis = 'y')

plt.show()

#### Observations
This is one very interesting factor to understand which group earns high income
* Asian-Pacific-Islander are a large chunk of people who have high income as well as less proportion of low income earners. Whites follow closely second

#### with respect to 'native_country'

In [228]:
# Grouping low income earners among different country origins

low_inc_cnt_country_wise = pd.DataFrame(low_income_df.groupby('native_country').size())
low_inc_cnt_country_wise.columns = ['low_income_count']
print(low_inc_cnt_country_wise.shape)
print(low_inc_cnt_country_wise.head(2))
print('\n\n')

# Grouping high income earners among different country origins

high_inc_cnt_country_wise = pd.DataFrame(high_income_df.groupby('native_country').size())
high_inc_cnt_country_wise.columns = ['high_income_count']
print(high_inc_cnt_country_wise.shape)
print(high_inc_cnt_country_wise.head(2))

# Merging the above two dataframes and arranging country-wise high income earners

inc_cnt_country_wise = low_inc_cnt_country_wise.merge(high_inc_cnt_country_wise, on = 'native_country', how = 'left')
inc_cnt_country_wise.fillna(0.0)
inc_cnt_country_wise['percent_low_income'] = ((inc_cnt_country_wise['low_income_count']*100)/(inc_cnt_country_wise['low_income_count']+inc_cnt_country_wise['high_income_count'])).round(2)
inc_cnt_country_wise['percent_high_income'] = ((inc_cnt_country_wise['high_income_count']*100)/(inc_cnt_country_wise['low_income_count']+inc_cnt_country_wise['high_income_count'])).round(2)
inc_cnt_country_wise.loc[:,['percent_low_income','percent_high_income']].sort_values('percent_high_income', ascending = False)

#### Observations
* 4 among the top 5 countries are Asian countries who earn significantly high income
* US natives have a ratio of 3:1 interms of low income to high income earners

#### with respect to 'age'

In [229]:
fig = plt.figure(figsize=(10,6))
sns.distplot(low_income_df.age, hist = False, label = 'Low Income')
sns.distplot(high_income_df.age, hist = False, label = 'High Income')
plt.legend()
plt.xticks([0,5,10,15,20,25,30,35,40,45,50,55,60,65,70,75,80,85,90,95,100])
plt.title("Distribution of age")
plt.grid(axis = 'x')
plt.show()

#### Observations
* Large chunk of low income earners lie in an age bracket of [15-35] while high income earners lie in an age bracket of [35-55]

#### with respect to 'education_num'

In [230]:
fig = plt.figure(figsize=(10,6))
sns.distplot(low_income_df.education_num, hist = False, label = 'Low Income')
sns.distplot(high_income_df.education_num, hist = False, label = 'High Income')
plt.legend()
plt.xlim([0,18])
plt.xticks([0,2,4,6,8,10,12,14,16,18])
plt.title("Distribution of number of years of education")
plt.grid(axis = 'x')
plt.show()

#### Observations
This graph also gives a very interesting insights
* Among people with less than 11 years of education are most likely to earn less than 50000 dollars/year, while those with more than 12 years of education earn more than 50000 dollars/year quite easily

#### with respect to 'hours_per_week'

In [231]:
fig = plt.figure(figsize=(10,6))
sns.distplot(low_income_df.hours_per_week, hist = False, label = 'Low Income')
sns.distplot(high_income_df.hours_per_week, hist = False, label = 'High Income')
plt.legend()
plt.xlim([0,110])
plt.xticks([0,5,10,15,20,25,30,35,40,45,50,55,60,65,70,75,80,85,90,95,100,105])
plt.title("Distribution of number of working hours per week")
plt.grid(axis = 'x')
plt.show()

#### Observations
* Large chunk of people work for an average of 40 hrs per week
* While those who work for more than 45 hrs/week are most likely to earn higher income

In [232]:
fig = plt.figure(figsize=(10,12))
sns.barplot(data = icd_df, x = 'age', y = 'occupation', hue = 'target', estimator = np.mean, palette='YlOrBr')
plt.xlim([23,55])
plt.grid(axis = 'x')
#plt.xticks([])
plt.show()

#### Observations
* Armed forces with an age of more than 30 years are most likely to earn an income higher than $50,000/year
* Among all the groups High income earners are elder than low income earners

In [233]:
fig = plt.figure(figsize=(10,12))
sns.barplot(data = icd_df, x = 'education_num', y = 'occupation', hue = 'target', estimator = np.mean, palette='YlOrBr')
plt.xlim([6,16])
plt.grid(axis = 'x')
#plt.xticks([])
plt.show()

#### Observations
* People with more 10/11 years of education are most likely high income earners
* People in executive managerial, professional speciality, armed forces and private-housing service roles have a large chunk of people with more than 12 years of education

In [234]:
icd_df.head()

#### Converting variables from categorical to numerical 

In [235]:
def binary_map1(x):
    return x.map({'Female': 1, "Male": 0})
icd_df[['sex']] = icd_df[['sex']].apply(binary_map1)

def binary_map2(x):
    return x.map({'>50K': 1, "<=50K": 0})
icd_df[['target']] = icd_df[['target']].apply(binary_map2)

In [236]:
# Check for data imbalance in target variables

icd_df.target.value_counts(normalize=True)*100

<a id="subsect-5"></a>
### Splitting the dataframe with respect to capital transactions and analysing

In [237]:
# Extracting the dataframe based on '0' capital gain/loss

zero_cap_df = icd_df[(icd_df.capital_gain == 0) & (icd_df.capital_loss == 0)]

# Extracting the dataframe based on those with significant capital gain/loss

gain_loss_cap_df = icd_df[(icd_df.capital_gain != 0) | (icd_df.capital_loss != 0)]

In [238]:
gain_loss_cap_df.head()

In [239]:
zero_cap_df.head()

In [240]:
col2drop = ['capital_gain','capital_loss']
zero_cap_df.drop(col2drop, axis = 1, inplace = True)

In [241]:
# Converting categorical variables into numerical variables

categ_cols = ['workclass','education','marital_status','occupation','relationship','race','native_country']

label_encoder = LabelEncoder()
for col in categ_cols:
    label_encoder.fit(zero_cap_df[col])
    zero_cap_df[col] = label_encoder.transform(zero_cap_df[col])
zero_cap_df.head()

In [242]:
# Check for class imbalance in target variable

zero_cap_df.target.value_counts(normalize=True)*100

In [243]:
# Converting categorical variables into numerical variables

categ_cols = ['workclass','education','marital_status','occupation','relationship','race','native_country']

label_encoder = LabelEncoder()
for col in categ_cols:
    label_encoder.fit(gain_loss_cap_df[col])
    gain_loss_cap_df[col] = label_encoder.transform(gain_loss_cap_df[col])
gain_loss_cap_df.head()

In [244]:
# Check for class imbalance in target variable

gain_loss_cap_df.target.value_counts(normalize=True)*100

##### Class imbalance is observed in zero_cap_df, while class imbalance is insignificant in gain_loss_cap_df

<a id="subsect-6"></a>
### Correlation Analysis

In [245]:
plt.figure(figsize=(12,12))

msk = np.array(zero_cap_df.corr())

msk[np.tril_indices_from(msk)] = False

sns.heatmap(zero_cap_df.corr(),mask=msk,annot=True,cmap='Greens')
plt.show()

In [246]:
plt.figure(figsize=(12,12))

msk = np.array(gain_loss_cap_df.corr())

msk[np.tril_indices_from(msk)] = False

sns.heatmap(gain_loss_cap_df.corr(),mask=msk,annot=True,cmap='Greens')
plt.show()

#### Observations
* Multicollinearity effects are significantly less both the dataframes

<a id="section-5"></a>
## Building the ML model on zero_cap_df [for those with zero capital transactions]

#### Test-Train split

In [247]:
# Test train split

X = zero_cap_df.drop('target', axis=1)
y = zero_cap_df['target']

# Splitting the data into train and test
X_tr, X_te, y_tr, y_te  = train_test_split(X,y, train_size=0.7, test_size=0.3, random_state=100, stratify=y )

In [248]:
print('Number of low income individuals in train: ',y_tr.sum())
print('Number of high income individuals in test: ',y_te.sum())
print('\nlow income to high income rate for train:', round(100*y_tr.sum()/len(y_tr),2))
print('low income to high income rate for test:', round(100*y_te.sum()/len(y_te),2))

In [249]:
print(X_tr.shape)
print(X_te.shape)
type(X_tr)

#### Feature Scaling

In [250]:
num_cols = list(X_tr.describe().columns)

sc = StandardScaler()
X_tr[num_cols] = sc.fit_transform(X_tr[num_cols])
X_te[num_cols] = sc.transform(X_te[num_cols])
X_tr.head()

<a id="subsect-7"></a>
### Model building with Recursive Feature Elimination

##### Initially we define some user-defined functions to build the model using logistic regression, checking for variance inflation factor, calculating optimal cutoff/ threshold probability for sigmoid classification and for "model report and metrics" 

In [251]:
def build_model(x1,y1):
    x2 = sm.add_constant(x1)                                      # Adding the constant
    logm1 = sm.GLM(y1, x2, family = sm.families.Binomial()).fit() # fitting the model
    print(logm1.summary())                                           # Model summary
    return x2,logm1
       
def checkVIF(X):
    vif = pd.DataFrame()
    vif['Features'] = X.columns
    vif['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    vif['VIF'] = round(vif['VIF'], 2)
    vif = vif.sort_values(by = "VIF", ascending = False)
    return(vif)

In [252]:
def opti_cutoff(y_actual, pred_proba):
    
    # Create 'pred_df' 
    pred_df = pd.DataFrame({'actual':y_actual.values, 'predicted_prob':pred_proba})
    
    # Churn_predicted for different cutoff probabilities 
    n = [float(x)/10 for x in range(10)]
    for i in n:
        pred_df[i]= pred_df.predicted_prob.map(lambda x: 1 if x > i else 0)
   
    # Accuracy, Sensitivity and Specificity for different cutoff probabilities
    cutoff_df = pd.DataFrame( columns = ['probability_cutoff','Accuracy','Sensitivity','Specificity'])
    for i in n:
        conf_mat = confusion_matrix(pred_df.actual, pred_df[i] )
        total = sum(sum(conf_mat))
        accuracy = (conf_mat[0,0]+conf_mat[1,1])/total
        specificity = conf_mat[0,0]/(conf_mat[0,0]+conf_mat[0,1])
        sensitivity = conf_mat[1,1]/(conf_mat[1,0]+conf_mat[1,1])
        cutoff_df.loc[int(i*10)] =[ i ,accuracy,sensitivity,specificity]
    print(cutoff_df)
    
    # Plot of Accuracy, Sensitivity and Specificity for different cutoff probabilities
    fig = plt.figure(figsize=[12,8])
    cutoff_df.plot.line(x='probability_cutoff', y=['Accuracy','Sensitivity','Specificity'])
    plt.xticks(np.linspace(0,1,21).round(4),fontsize=10,rotation = 50)
    plt.grid(axis = 'x')
    plt.show()
    
    return pred_df

In [253]:
def model_report(y_actual, y_predicted):
    
    # Confusion matrix 
    confusion = confusion_matrix(y_actual, y_predicted)
    print('confusion matrix: \n',confusion)
    
    TP = confusion[1,1] # true positive 
    TN = confusion[0,0] # true negatives
    FP = confusion[0,1] # false positives
    FN = confusion[1,0] # false negatives
    
    # Accuracy score of the model: (Not suitable in case of imbalanced dataset, accuracy gives higher weightage to majority class)
    print("\nAccuracy = " , round(accuracy_score(y_actual, y_predicted)*100,4))
    print('Accuracy is indicating the ratio of correct predictions to the total number of predictions')
    
    # Sensitivity of the model:
    print('\nSensitivity:   ', round(100*TP / float(TP+FN),4))
    print('Sensitivity is indicating the ratio of correctly predicted churn cases to the total churn cases')

    # Specificity of the model:
    print('\nSpecificity:   ', round(100*TN / float(TN+FP),4))
    print('Specificity is indicating the ratio of correctly predicted non-churn prospects to the total non-churn cases')

    # Precision is a good measure to determine accuracy of the model, when the costs of False Positive is high 
    # (i.e., prediction of positives should be as accurate as possible). Preferably helpful in case of imbalanced data as well.
    # Example: Email spam detection - you don't want a non-spam email be detected as spam which leads to missing important mail.
    print("\nPrecision = " ,round(precision_score(y_actual, y_predicted)*100,4))
    
    #Recall is a metric used to select our best model when there is a high cost associated with False Negative
    # Best metric when the costs of False Negative is high (i.e., prediction of positives should be as accurate as possible)
    # Example:  Fraud detection or sick patient detection - you don't want a sick patient to be detected as not sick.
    print("Recall = " ,round(recall_score(y_actual, y_predicted)*100,4))
    
    # F1-score gives a balance between precision and recall, suitable for imbalanced class datasets
    print("F1 Score = " ,round(f1_score(y_actual, y_predicted)*100,4))
    print('\nAll metrics measured in terms of %')
    pass

In [254]:
def fn_roc_curve(y_actual, pred_proba):
    
    fpr, tpr, thresholds = roc_curve(y_actual,  pred_proba, drop_intermediate = False)
    auc_score = roc_auc_score( y_actual, pred_proba )
    plt.figure(figsize=(7, 7))
    plt.plot( fpr, tpr, label='Area under ROC curve AUC score = %0.4f' % (100*auc_score) )
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate or [1 - True Negative Rate]')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic (ROC curve)')
    plt.legend(loc="lower right")
    plt.show()
    
    pass

#### Feature elimination using RFECV (Recursive Feature Elimination with Cross Validation)

In [255]:
# Creating an object for logistic regression

logreg = LogisticRegression(class_weight='balanced') # // class_weight='balanced' \\ is to handle class imbalance in the dataset

In [256]:
rfecv = RFECV(estimator=logreg, cv=5, n_jobs=-1) # 5 fold cross validation is used
rfecv.fit(X_tr, y_tr)

In [257]:
plt.figure(figsize=[10, 5])
plt.plot(range(1, X_tr.shape[1]+1), rfecv.grid_scores_)
plt.xlabel('Number of Features')
plt.ylabel('')
plt.legend()
plt.xlim([0,13])
plt.grid('x')
plt.show()

#### Observations
The grid scores stabilize after 6 features

#### Number of features selected by RFECV

In [258]:
rfecv.n_features_

In [259]:
# Assigning RFE supported columns to new dataframe

col_rfecv = X_tr.columns[rfecv.support_]
X_tr_rfecv = X_tr[col_rfecv]
X_tr_rfecv.head()

In [260]:
X_tr.head()

native_country is dropped using RFECV

#### Assessing the model with StatsModels

In [261]:
X_tr_rfecv_const,logm1 = build_model(X_tr_rfecv, y_tr)

In [262]:
checkVIF(X_tr_rfecv_const)

##### From VIF, we get to know the insignificant multicollinearity effects, while all variables are found to be significant with a p -value of less than 0.055

#### Prediction and evaluation on train 

In [263]:
# Getting the Predicted values on the test dataset
y_tr_pred_proba = logm1.predict(X_tr_rfecv_const).values.reshape(-1)

#### Finding Optimal Cutoff Point

In [264]:
# Call function
y_pred_df = opti_cutoff(y_tr, y_tr_pred_proba)

# Output of the above function call will give threshold probability from the Accuracy-Sensitivity-Specificity trade-off plot

#### From the curve, (0.20-0.25) lies the optimum point, so 0.22 is taken as threshold probability based 'Sensitivity-Specificity' tradeoff

In [265]:
# Getting the 'y_pred_df.predicted' using threshold probability value

opti_prob = 0.22   # Get it from Accuracy-Sensitivity-Specificity trade-off plot
y_pred_df['predicted'] = y_pred_df.predicted_prob.apply( lambda x: 1 if x > opti_prob else 0) 

In [266]:
# Gives detailed classification report
model_report(y_tr, y_pred_df.predicted)

In [267]:
# Gives ROC curve
fn_roc_curve(y_tr, y_pred_df.predicted_prob)

<a id="subsect-8"></a>
### Random forest with RFE 

In [268]:
# Random Forest

params = {
          'max_depth': [3, 5, 10, 15, 20],
          'min_samples_leaf': [5, 10, 25, 50, 100, 300],
          'n_estimators': [10, 25, 50, 75, 100],
          'criterion': ["gini"]
         }

strat_cv = StratifiedKFold( n_splits=6, shuffle=True, random_state=40 )

rf_rfecv_gscv = GridSearchCV(
                    estimator = RandomForestClassifier(class_weight='balanced', random_state=42),
                    param_grid = params,
                    scoring = 'f1',
                    cv = strat_cv,
                    n_jobs=-1, verbose=1
                    )

rf_rfecv_gscv_res = rf_rfecv_gscv.fit(X_tr_rfecv, y_tr)
rf_rfecv_best_esti = rf_rfecv_gscv_res.best_estimator_
print(rf_rfecv_best_esti)

In [269]:
# Getting the Predicted values on the train dataset
y_tr_pred_probblty = rf_rfecv_best_esti.predict_proba(X_tr_rfecv)[:,1]#.values.reshape(-1)
y_tr_predict = rf_rfecv_best_esti.predict(X_tr_rfecv)

# Getting X_test df
X_te_rfecv = X_te[col_rfecv]

# Getting the Predicted values on the test dataset
y_te_pred_probblty = rf_rfecv_best_esti.predict_proba(X_te_rfecv)[:,1]#.values.reshape(-1)
y_te_predict = rf_rfecv_best_esti.predict(X_te_rfecv)

#### Evaluation on train dataset

In [270]:
# Gives detailed classification report
model_report(y_tr, y_tr_predict)

#### Evaluation on test dataset

In [271]:
# Gives detailed classification report
model_report(y_te, y_te_predict)

#### ROC curve on train data

In [272]:
# Gives ROC curve
fn_roc_curve(y_tr, y_tr_pred_probblty)

#### ROC curve on test data

In [273]:
# Gives ROC curve
fn_roc_curve(y_te, y_te_pred_probblty)

### Feature Importance

In [274]:
imp_df = pd.DataFrame({
                        "Features": X_tr_rfecv.columns,
                        "Importance": rf_rfecv_best_esti.feature_importances_
                      })

print('Top 10 features by importance are:\n')
top10_df = imp_df.sort_values(by="Importance", ascending=False).head(10)

plt.figure(figsize=(8,10))
sns.barplot(x="Importance", y="Features", data= top10_df, palette = "Greens_r")
plt.show()

<a id="subsect-9"></a>
### Model building with Principal Component Analysis (PCA)

In [275]:
modelpca = PCA(random_state=42)
pca = modelpca.fit(X_tr)

var_cumu = np.cumsum(pca.explained_variance_ratio_)

fig = plt.figure(figsize=[12,8])
plt.vlines(x=10, ymax=1.05, ymin=0, colors="r", linestyles="--")
plt.ylim([0,1.05])
plt.xlim([0,13])
plt.hlines(y=0.95, xmax=80, xmin=0, colors="g", linestyles="--")
plt.plot(var_cumu)
plt.ylabel("Cumulative variance explained")
plt.show()

##### 10 features explain more than 95% of  variance, hence 'n_components' is taken as 10

In [276]:
pca_final = IncrementalPCA(n_components=10)
pca_train = pca_final.fit_transform(X_tr)

df_train_pca = pd.DataFrame(pca_train)
df_train_pca.head()

In [277]:
pca_test = pca_final.transform(X_te)

df_test_pca = pd.DataFrame(pca_test)
df_test_pca.shape

In [278]:
corrmat = np.corrcoef(df_train_pca.transpose())

plt.figure(figsize=[10,10])
sns.heatmap(corrmat, annot=True)

<a id="subsect-10"></a>
### Random forest with PCA

In [279]:
# Random Forest

params = {
          'max_depth': [3, 5, 10, 15, 20],
          'min_samples_leaf': [5, 10, 25, 50, 100, 300],
          'n_estimators': [10, 25, 50, 75, 100],
          'criterion': ["gini"]
         }

strat_cv = StratifiedKFold( n_splits=6, shuffle=True, random_state=40 )

rf_pca_gscv = GridSearchCV(
                    estimator = RandomForestClassifier(class_weight='balanced', random_state=42),
                    param_grid = params,
                    scoring = 'f1',
                    cv = strat_cv,
                    n_jobs=-1, verbose=1
                    )

rf_pca_gscv_res = rf_pca_gscv.fit(df_train_pca, y_tr)
rf_pca_best_esti = rf_pca_gscv_res.best_estimator_
print(rf_pca_best_esti)

In [280]:
# Getting the Predicted values on the train dataset
y_tr_pred_probblty = rf_pca_best_esti.predict_proba(df_train_pca)[:,1]#.values.reshape(-1)
y_tr_predict = rf_pca_best_esti.predict(df_train_pca)

# Getting the Predicted values on the test dataset
y_te_pred_probblty = rf_pca_best_esti.predict_proba(df_test_pca)[:,1]#.values.reshape(-1)
y_te_predict = rf_pca_best_esti.predict(df_test_pca)

#### Evaluation on train dataset

In [281]:
# Gives detailed classification report
model_report(y_tr, y_tr_predict)

#### Evaluation on test dataset

In [282]:
# Gives detailed classification report
model_report(y_te, y_te_predict)

#### ROC curve on train data

In [283]:
# Gives ROC curve
fn_roc_curve(y_tr, y_tr_pred_probblty)

#### ROC curve on test data

In [284]:
# Gives ROC curve
fn_roc_curve(y_te, y_te_pred_probblty)

<a id="subsect-11"></a>
### Gradient boosting method with RFE

In [285]:
# Tuning 'n_estimators' initially in order to reduce the running time with other hyperparameters 

params = {
        'n_estimators': [3,5,7,9],
        'max_depth': [3, 5, 10, 15, 20],
        'min_samples_leaf': [5, 10, 20, 50, 100]
        }

strat_cv = StratifiedKFold( n_splits=10, shuffle=True, random_state=40 )

gbmn_gscv = GridSearchCV(
                    estimator = GradientBoostingClassifier(max_features = 10,learning_rate = 0.1,max_depth = 5,random_state = 45),
                    param_grid = params,
                    scoring = 'f1',
                    cv = strat_cv,
                    n_jobs=-1, verbose=1
                    )

gbmn_gscv_res = gbmn_gscv.fit(X_tr_rfecv, y_tr)
gbmn_best_esti = gbmn_gscv_res.best_estimator_
print(gbmn_best_esti)


In [286]:
print(gbmn_gscv_res.best_params_) # This best 'n_estimators' will be taken for further tuning

In [287]:
# Getting the Predicted values on the train dataset
y_tr_pred_probblty = gbmn_best_esti.predict_proba(X_tr_rfecv)[:,1]#.values.reshape(-1)
y_tr_predict = gbmn_best_esti.predict(X_tr_rfecv)

# Getting X_test df
X_te_rfecv = X_te[col_rfecv]

# Getting the Predicted values on the test dataset
y_te_pred_probblty = gbmn_best_esti.predict_proba(X_te_rfecv)[:,1]#.values.reshape(-1)
y_te_predict = gbmn_best_esti.predict(X_te_rfecv)

#### Evaluation on train dataset

In [288]:
# Gives detailed classification report
model_report(y_tr, y_tr_predict)

#### Evaluation on test dataset

In [289]:
# Gives detailed classification report
model_report(y_te, y_te_predict)

#### ROC curve on train data

In [290]:
# Gives ROC curve
fn_roc_curve(y_tr, y_tr_pred_probblty)

#### ROC curve on test data

In [291]:
# Gives ROC curve
fn_roc_curve(y_te, y_te_pred_probblty)

#### Model comparison and conclusion is discussed at the end

<a id="section-6"></a>
## Building the ML model on gain_loss_cap_df [for those with some significant capital transactions]

#### Test-Train split

In [292]:
# Test train split

X = gain_loss_cap_df.drop('target', axis=1)
y = gain_loss_cap_df['target']

# Splitting the data into train and test
X_tr, X_te, y_tr, y_te  = train_test_split(X,y, train_size=0.7, test_size=0.3, random_state=100, stratify=y )

print('Number of low income individuals in train: ',y_tr.sum())
print('Number of high income individuals in test: ',y_te.sum())
print('\nlow income to high income rate for train:', round(100*y_tr.sum()/len(y_tr),2))
print('low income to high income rate for test:', round(100*y_te.sum()/len(y_te),2))
print(X_tr.shape)
print(X_te.shape)

#### Feature Scaling

In [293]:
num_cols = list(X_tr.describe().columns)

sc = StandardScaler()
X_tr[num_cols] = sc.fit_transform(X_tr[num_cols])
X_te[num_cols] = sc.transform(X_te[num_cols])
X_tr.head()

#### Feature elimination using RFECV

In [294]:
# Creating an object for logistic regression

logreg = LogisticRegression(class_weight='balanced') # // class_weight='balanced' \\ is to handle class imbalance in the dataset

In [295]:
rfecv = RFECV(estimator=logreg, cv=5, n_jobs=-1)
rfecv.fit(X_tr, y_tr)

In [296]:
plt.figure(figsize=[10, 5])
plt.plot(range(1, X_tr.shape[1]+1), rfecv.grid_scores_)
plt.xlabel('Number of Features')
plt.ylabel('RFECV Grid scores')
plt.xlim([0,13])
plt.grid('x')
plt.show()

#### Number of features selected by RFECV

In [297]:
rfecv.n_features_

In [298]:
# Assigning RFE supported columns to new dataframe

col_rfecv = X_tr.columns[rfecv.support_]
X_tr_rfecv = X_tr[col_rfecv]
X_tr_rfecv.head()

In [299]:
X_tr.head()

#### Random forest gave the best result for the first dataframe zero_cap_df, hence this model is built using random forest with RFECV 

<a id="subsect-12"></a>
### Random forest with RFE 

In [300]:
# Random Forest

params = {
          'max_depth': [3, 5, 10, 15, 20],
          'min_samples_leaf': [5, 10, 25, 50, 100, 300],
          'n_estimators': [10, 25, 50, 75, 100],
          'criterion': ["gini"]
         }

strat_cv = StratifiedKFold( n_splits=6, shuffle=True, random_state=40 )

rf_rfecv_gscv = GridSearchCV(
                    estimator = RandomForestClassifier(class_weight='balanced', random_state=42),
                    param_grid = params,
                    scoring = 'f1',
                    cv = strat_cv,
                    n_jobs=-1, verbose=1
                    )

rf_rfecv_gscv_res = rf_rfecv_gscv.fit(X_tr_rfecv, y_tr)
rf_rfecv_best_esti = rf_rfecv_gscv_res.best_estimator_
print(rf_rfecv_best_esti)

In [301]:
# Getting the Predicted values on the train dataset
y_tr_pred_probblty = rf_rfecv_best_esti.predict_proba(X_tr_rfecv)[:,1]#.values.reshape(-1)
y_tr_predict = rf_rfecv_best_esti.predict(X_tr_rfecv)

# Getting X_test df
X_te_rfecv = X_te[col_rfecv]

# Getting the Predicted values on the test dataset
y_te_pred_probblty = rf_rfecv_best_esti.predict_proba(X_te_rfecv)[:,1]#.values.reshape(-1)
y_te_predict = rf_rfecv_best_esti.predict(X_te_rfecv)

#### Evaluation on train dataset

In [302]:
# Gives detailed classification report
model_report(y_tr, y_tr_predict)

#### Evaluation on test dataset

In [303]:
# Gives detailed classification report
model_report(y_te, y_te_predict)

#### ROC curve on train data

In [304]:
# Gives ROC curve
fn_roc_curve(y_tr, y_tr_pred_probblty)

#### ROC curve on test data

In [305]:
# Gives ROC curve
fn_roc_curve(y_te, y_te_pred_probblty)

### Feature Importance

In [306]:
imp_df = pd.DataFrame({
                        "Features": X_tr_rfecv.columns,
                        "Importance": rf_rfecv_best_esti.feature_importances_
                      })

print('Top 10 features by importance are:\n')
top10_df = imp_df.sort_values(by="Importance", ascending=False).head(10)

plt.figure(figsize=(8,10))
sns.barplot(x="Importance", y="Features", data= top10_df, palette = "Greens_r")
plt.show()

<a id="section-7"></a>
## Conclusions
#### All the objectives has been satisfactorily achieved. 

### Results
#### Dataset was split into two dataframes and analyzed with respect to capital transactions
#### zero_cap_df - with zero capital transactions
#### As the data was having class imbalance 'F1-score' is used for evaluations, as it is considered as the best metric for comparison between different models, though other metrics were also verified. 
#### Also evaluation is focussed on results obtained by the model on test data set, while the test-train comparison is also done for a couple of models
#### Model Comparison                                                                                                                           Test F1-score
##### 1. Logistic Regression with RFECV   ----------------------------------------------------------------------->     53.8 (train)
##### 2. Random Forest with RFECV  ------------------------------------------------------------------------------->    61.1    <---  Best score
##### 3. Random Forest with PCA data  --------------------------------------------------------------------------->    60.6
##### 4. Gradient boost method with RFECV -------------------------------------------------------------------->    42.5

#### gain_loss_cap_df - with significant capital transactions
#####  Random Forest with RFECV  ------------------------------------------------------------------------------->     93.5

### Observations
##### * relationship status/ marital status, age, number of years of education are among the top features that helps significantly to predict high income customers for those with nil capital transactions
##### * capital gain/loss,number of educational years, marital status and age are among the top features that helps significantly to predict high income customers for those with some significant capital transactions